In [25]:
import random


class Unit:
    def __init__(self, name):
        self.name = name

class Employee:
    def __init__(self, name, unit, priority: bool = False):
        self.name = name
        self.unit = unit
        self.priority = priority

    def sayName(self):
        print(self.name)

    def sayUnit(self):
        print(self.unit)



#stack is used to determine the order of bookings and if a room is full
class Stack:
    def __init__(self, max_size):
        self.max_size = max_size
        self.stack = []
        self.size = 0

    def is_full(self):
        return len(self.stack) == self.max_size

    def push(self, item):
        if not self.is_full():
            self.stack.append(item)
            self.size += 1
        else:
            print("Stack is full")

    def pop(self):
        if not self.is_empty():
            self.size -= 1
            return self.stack.pop()
        else:
            print("Stack is empty")
    def is_empty(self):
        return len(self.stack) == 0

    def size(self):
        return len(self.stack)

class Room:
    def __init__(self, name, n_desks):
        self.name = name
        self.desks = Stack(n_desks)
        self.attendingUnits = []


    def isFull(self):
        return self.desks.is_full()

    def isEmpty(self):
        return self.desks.size == 0

    def placeAttendee(self, attendee):
        if not self.isFull():
            self.desks.push(attendee)
            print(f"{attendee.name} has been placed in {self.name}. Still {(self.desks.max_size - self.desks.size)} desks left")
            self.attendingUnits.append(attendee.unit)
        else:
            print("Room is full")

    def removeAttendee(self, attendee):
        if attendee in self.desks.stack:
            self.desks.stack.remove(attendee)
            print(f"{attendee.name} has been removed from {self.name}")
            self.attendingUnits.remove(attendee.unit)
        else:
            print(f"{attendee.name} is not in {self.name}")



    def getAttendingUnits(self):
        return self.attendingUnits






def chooseRandomPeople(employees):
    ''' Choose a random number of people from the employees list '''
    num_people = random.randint(1, len(employees))
    attendees = random.sample(employees, num_people)

    return attendees



def createDayExample(employees):
    ''' Create a day example '''
    attendees = chooseRandomPeople(employees)


    return attendees




In [7]:
employees = [
    Employee("Nick", "Dev", True),
    Employee("Florian", "Dev", False),
    Employee("Oli", "Dev", False),
    Employee("Dominik", "Dev", False),
    Employee("Rick", "Dev", True),
    Employee("Sönke", "Dev", False),
    Employee("Timo", "Dev", True),

    Employee("Alno", "PO", True),
    Employee("Erkan", "PO", False),

    Employee("Till", "Marketing", False),
    Employee("Lukas", "Marketing", False),
    Employee("Marie", "Marketing", False),
    Employee("Marieke", "Marketing", False),
    Employee("Lilly", "Marketing", False),
    Employee("Birte", "Marketing", False),

    Employee("Johannes", "UX", False ),
    Employee("Julia", "UX", False),
    Employee("Elena", "UX", True),

    Employee("Artur", "CS", False),
    Employee("Phillip", "CS", False),
    Employee("Tony", "CS", False),
    Employee("Monana", "CS", False),

    Employee("Karim", "Sales", False),
    Employee("Marei", "Sales", False),
    Employee("Angelika", "Sales", False),
    Employee("Else", "Sales", False),
    Employee("Jannik", "Sales", False),

    ]

# unitMatches shows which units can sit in the same room, sorted by best match
unitMatches = {
    "Dev": ["PO", "UX", "Marketing"],
    "PO":["Dev","UX","Marketing"],
    "Marketing": ["PO", "UX", "Dev"],
    "UX": ["PO","Dev","Marketing"],
    "CS": ["Sales"],
    "Sales": ["CS"]
}



In [15]:
attendees = createDayExample(employees=employees)
rooms = [Room("Aquarium", 4), Room("BigMamba", 10), Room("Loge", 6), Room("Cave", 6)]

for attendee in attendees:
    print(attendee.name, attendee.unit)


Phillip CS
Johannes UX
Timo Dev
Jannik Sales
Else Sales
Till Marketing
Nick Dev
Elena UX
Erkan PO
Lukas Marketing
Birte Marketing
Monana CS
Angelika Sales


In [23]:
# recursive function to place all attendees

def placeAttendees(rooms, attendees):
    # start from first attendee
    if not attendees:
        return True
    person = attendees.pop(0)
    print(f"Placing {person.name} in a room")
    # check if there is a room with an attendee from the same unit
    sameUnitRooms = [room for room in rooms if person.unit in room.attendingUnits and not room.isFull()]
    sameUnitRoomsNames = [room.name for room in sameUnitRooms]

    while sameUnitRooms:
        print(f"Rooms with same unit: {sameUnitRoomsNames} as {person.name}s unit")
        # as long as there are rooms with attendees from the same unit, place the person in the first room
        print(f"Placing {person.name} in the room {sameUnitRooms[0].name}")
        sameUnitRooms[0].placeAttendee(person)
        if placeAttendees(rooms, attendees):
            break

        else:
            print(f"Removing {person.name} from the room {sameUnitRooms[0].name}")
            removedRoom = sameUnitRooms.pop(0)
            removedRoom.removeAttendee(person)




    else:
        # check if there is an empty room available
        print(f"Trying to place {person.name} in an empty room")
        emptyRooms = [room for room in rooms if room.isEmpty()]
        print(f"There are {len(emptyRooms)} for {person.name}")
        while emptyRooms:
            print(f"Placing {person.name} in the room {emptyRooms[0].name}")
            emptyRooms[0].placeAttendee(person)
            if placeAttendees(rooms, attendees):
                break
            else:
                print(f"Removing {person.name} from the room {emptyRooms[0].name}")
                removedRoom = emptyRooms.pop(0)
                removedRoom.removeAttendee(person)



        else: # no empty rooms available. Now we look for rooms with matching units
            # 1. get all rooms with the best matching units
            matchingUnits = unitMatches[person.unit]
            print(f"{person.name} has these matching Units: {matchingUnits}")
            matchingUnitRooms = []
            for room in rooms:
                if all(unit in matchingUnits for unit in room.attendingUnits) and not room.isFull():
                    matchingUnitRooms.append(room)
            while matchingUnitRooms:
                print(f"There is a room with matching units. Placing {person.name} in the room {matchingUnitRooms[0].name}")
                matchingUnitRooms[0].placeAttendee(person)
                if placeAttendees(rooms, attendees):
                    break
                else:
                    print(f"Removing {person.name} from the room {matchingUnitRooms[0].name}")
                    removedRoom = matchingUnitRooms.pop(0)
                    removedRoom.removeAttendee(person)
            else:
                print("No suitable room found for the person")
                # add person to the list again
                attendees.append(person)
                # move the last person to another room
                return False

In [10]:
placeAttendees(rooms, attendees)

Placing Marei in a room
Rooms with same unit: [] as Mareis unit
Trying to place Marei in an empty room
There are 4 for Marei
Placing Marei in the room Aquarium
Marei has been placed in Aquarium. Still 3 desks left
Placing Rick in a room
Rooms with same unit: [] as Ricks unit
Trying to place Rick in an empty room
There are 3 for Rick
Placing Rick in the room BigMamba
Rick has been placed in BigMamba. Still 9 desks left
Placing Julia in a room
Rooms with same unit: [] as Julias unit
Trying to place Julia in an empty room
There are 2 for Julia
Placing Julia in the room Loge
Julia has been placed in Loge. Still 5 desks left
Placing Monana in a room
Rooms with same unit: [] as Monanas unit
Trying to place Monana in an empty room
There are 1 for Monana
Placing Monana in the room Cave
Monana has been placed in Cave. Still 5 desks left
Placing Karim in a room
Rooms with same unit: ['Aquarium'] as Karims unit
Placing Karim in the room Aquarium
Karim has been placed in Aquarium. Still 2 desks le

In [11]:
for room in rooms:
    print(f"Room {room.name} has the following units: {room.getAttendingUnits()}")
    print(f"And the follwoing attendees")
    for desk in room.desks.stack:
        print(desk.name)

Room Aquarium has the following units: []
And the follwoing attendees
Room BigMamba has the following units: ['Marketing', 'Sales']
And the follwoing attendees
Lukas
Marei
Room Loge has the following units: ['Dev', 'Marketing', 'Dev']
And the follwoing attendees
Timo
Marie
Rick
Room Cave has the following units: ['Sales', 'UX']
And the follwoing attendees
Karim
Julia


In [20]:
test = Room("Test", 4)
nick = Employee("Nick", "Dev", True)

test.placeAttendee(nick)

for desk in test.desks.stack:
    print(desk.name)

Nick has been placed in Test. Still 1 desks left
Nick


In [26]:
testRooms = [Room("Test1", 2), Room("Test2", 4)]
testAttendees = [Employee("Nick", "Dev", True), Employee("Florian", "PO", False), Employee("Oli", "Dev", False), Employee("Karim", "Sales", False)]

placeAttendees(testRooms, testAttendees)

Placing Nick in a room
Trying to place Nick in an empty room
There are 2 for Nick
Placing Nick in the room Test1
Nick has been placed in Test1. Still 1 desks left
Placing Florian in a room
Trying to place Florian in an empty room
There are 1 for Florian
Placing Florian in the room Test2
Florian has been placed in Test2. Still 3 desks left
Placing Oli in a room
Rooms with same unit: ['Test1'] as Olis unit
Placing Oli in the room Test1
Oli has been placed in Test1. Still 0 desks left
Placing Karim in a room
Trying to place Karim in an empty room
There are 0 for Karim
Karim has these matching Units: ['CS']
No suitable room found for the person
Removing Oli from the room Test1
Oli has been removed from Test1
Trying to place Oli in an empty room
There are 0 for Oli
Oli has these matching Units: ['PO', 'UX', 'Marketing']
There is a room with matching units. Placing Oli in the room Test2
Oli has been placed in Test2. Still 2 desks left
Placing Karim in a room
Trying to place Karim in an empty

False

In [27]:
for room in testRooms:
    print(f"Room {room.name} has the following units: {room.getAttendingUnits()}")
    print(f"And the follwoing attendees")
    for desk in room.desks.stack:
        print(desk.name)

Room Test1 has the following units: []
And the follwoing attendees
Room Test2 has the following units: ['PO', 'Dev']
And the follwoing attendees
Florian
Oli


In [4]:
begrüßung = 12

for i in range(begrüßung):

    print("Hallo Caren! Ich hoffe es geht dir gut!")

Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
Hallo Caren! Ich hoffe es geht dir gut!
